## Step 1: Import Packages and Declare Constants

In [17]:
import boto3
import sagemaker
import datetime as dt
import pandas as pd

In [18]:
#Replace this value with the S3 Bucket Created
default_bucket = "customer-churn-sm-pipeline-joshua-aws"

In [19]:
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
sklearn_processor_version="0.23-1"
model_package_group_name="ChurnModelPackageGroup"
pipeline_name= "ChurnModelSMPipeline"
clarify_image = sagemaker.image_uris.retrieve(framework='sklearn',version=sklearn_processor_version,region=region)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.


## Step 2: Generate Baseline Dataset

Baseline Data will be used as part of SageMaker Clarify Step to generate SHAP Values 

In [20]:
def preprocess_data(file_path):
    df = pd.read_csv(file_path)
    
    ## Convert to datetime columns
    df["firstorder"]=pd.to_datetime(df["firstorder"],errors='coerce')
    df["lastorder"] = pd.to_datetime(df["lastorder"],errors='coerce')
    ## Drop Rows with null values
    df = df.dropna()
    ## Create Column which gives the days between the last order and the first order
    df["first_last_days_diff"] = (df['lastorder']-df['firstorder']).dt.days
    ## Create Column which gives the days between when the customer record was created and the first order
    df['created'] = pd.to_datetime(df['created'])
    df['created_first_days_diff']=(df['created']-df['firstorder']).dt.days
    ## Drop Columns
    df.drop(['custid','created','firstorder','lastorder'],axis=1,inplace=True)
    ## Apply one hot encoding on favday and city columns
    
    df = pd.get_dummies(df,prefix=['favday','city'],columns=['favday','city'])
    # Replace 'True'/'False' strings and boolean values with 1/0
    df.replace({'True': 1, 'False': 0, True: 1, False: 0}, inplace=True)
    
    return df

In [21]:
baseline_data = preprocess_data("data/storedata_total.csv")
baseline_data.pop("retained")
baseline_sample = baseline_data.sample(frac=0.0002)

/tmp/ipykernel_165/3904479252.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'True': 1, 'False': 0, True: 1, False: 0}, inplace=True)


In [22]:
pd.DataFrame(baseline_sample).to_csv("data/baseline.csv",header=False,index=False)

## Step 3: Generate Batch Dataset

In [23]:
batch_data = preprocess_data("data/storedata_total.csv")
batch_data.pop("retained")
batch_sample = batch_data.sample(frac=0.2)

/tmp/ipykernel_165/3904479252.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'True': 1, 'False': 0, True: 1, False: 0}, inplace=True)


In [24]:
pd.DataFrame(batch_sample).to_csv("data/batch.csv",header=False,index=False)

## Step 4: Copy Data and Scripts to S3 Bucket

In [25]:
s3_client = boto3.resource('s3')
s3_client.Bucket(default_bucket).upload_file("data/storedata_total.csv","data/storedata_total.csv")
s3_client.Bucket(default_bucket).upload_file("data/batch.csv","data/batch/batch.csv")
s3_client.Bucket(default_bucket).upload_file("data/baseline.csv","input/baseline/baseline.csv")

In [26]:
s3_client.Bucket(default_bucket).upload_file("pipelines/customerchurn/preprocess.py","input/code/preprocess.py")
s3_client.Bucket(default_bucket).upload_file("pipelines/customerchurn/evaluate.py","input/code/evaluate.py")
s3_client.Bucket(default_bucket).upload_file("pipelines/customerchurn/generate_config.py","input/code/generate_config.py")

## Step 5: Get the Pipeline Instance

In [27]:
from pipelines.customerchurn.pipeline import get_pipeline

pipeline = get_pipeline(
    region = region,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
    custom_image_uri=clarify_image,
    sklearn_processor_version=sklearn_processor_version
)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.0.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [28]:
pipeline.definition()

'{"Version": "2020-12-01", "Metadata": {}, "Parameters": [{"Name": "ProcessingInstanceType", "Type": "String", "DefaultValue": "ml.m5.xlarge"}, {"Name": "ProcessingInstanceCount", "Type": "Integer", "DefaultValue": 1}, {"Name": "TrainingInstanceType", "Type": "String", "DefaultValue": "ml.m5.xlarge"}, {"Name": "InputData", "Type": "String", "DefaultValue": "s3://customer-churn-sm-pipeline-joshua-aws/data/storedata_total.csv"}, {"Name": "BatchData", "Type": "String", "DefaultValue": "s3://customer-churn-sm-pipeline-joshua-aws/data/batch/batch.csv"}], "PipelineExperimentConfig": {"ExperimentName": {"Get": "Execution.PipelineName"}, "TrialName": {"Get": "Execution.PipelineExecutionId"}}, "Steps": [{"Name": "ChurnModelProcess", "Type": "Processing", "Arguments": {"ProcessingResources": {"ClusterConfig": {"InstanceType": {"Get": "Parameters.ProcessingInstanceType"}, "InstanceCount": {"Get": "Parameters.ProcessingInstanceCount"}, "VolumeSizeInGB": 30}}, "AppSpecification": {"ImageUri": "6833

## Step 5: Submit the pipeline to SageMaker and start execution

In [29]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:476114112446:pipeline/ChurnModelSMPipeline',
 'ResponseMetadata': {'RequestId': '29055bc8-82d5-48aa-800d-5096c4ca1983',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '29055bc8-82d5-48aa-800d-5096c4ca1983',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '88',
   'date': 'Sun, 19 Jan 2025 22:51:52 GMT'},
  'RetryAttempts': 0}}

Start Pipeline Execution

In [30]:
execution = pipeline.start()

Now we describe execution instance and list the steps in the execution to find out more about the execution.

In [31]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:476114112446:pipeline/ChurnModelSMPipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:476114112446:pipeline/ChurnModelSMPipeline/execution/ptbgtjc1o4oj',
 'PipelineExecutionDisplayName': 'execution-1737327115007',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'churnmodelsmpipeline',
  'TrialName': 'ptbgtjc1o4oj'},
 'CreationTime': datetime.datetime(2025, 1, 19, 22, 51, 54, 936000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2025, 1, 19, 22, 51, 54, 936000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:476114112446:user-profile/d-zb9pfqdauvg4/default-20250115T135317',
  'UserProfileName': 'default-20250115T135317',
  'DomainId': 'd-zb9pfqdauvg4',
  'IamIdentity': {'Arn': 'arn:aws:sts::476114112446:assumed-role/AmazonSageMaker-ExecutionRole-20250115T135318/SageMaker',
   'PrincipalId': 'AROAW5WU4Z67M4PU73CNU:SageMaker'}},
 'LastModifiedBy':

We can list the execution steps to check out the status and artifacts:

In [34]:
execution.list_steps()

[{'StepName': 'ClarifyProcessingStep',
  'StartTime': datetime.datetime(2025, 1, 19, 23, 2, 42, 207000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2025, 1, 19, 23, 20, 16, 509000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:476114112446:processing-job/pipelines-ptbgtjc1o4oj-ClarifyProcessingSte-Lhbccyy8Pl'}},
  'AttemptCount': 1},
 {'StepName': 'ChurnModelConfigFile',
  'StartTime': datetime.datetime(2025, 1, 19, 23, 0, 5, 639000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2025, 1, 19, 23, 2, 41, 540000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:476114112446:processing-job/pipelines-ptbgtjc1o4oj-ChurnModelConfigFile-9jM8ZPTFQN'}},
  'AttemptCount': 1},
 {'StepName': 'ChurnTransform',
  'StartTime': datetime.datetime(2025, 1, 19, 23, 0, 5, 639000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2025, 1, 19, 23, 5, 22, 384000,